In [5]:
%load_ext autoreload

import torch
import numpy as np
from kel.estimation import estimation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate some data

In [6]:
def generate_data(n_sample):
    e = np.random.normal(loc=0, scale=1.0, size=[n_sample, 1])
    gamma = np.random.normal(loc=0, scale=0.1, size=[n_sample, 1])
    delta = np.random.normal(loc=0, scale=0.1, size=[n_sample, 1])

    z = np.random.uniform(low=-3, high=3, size=[n_sample, 1])
    t = np.reshape(z[:, 0], [-1, 1]) + e + gamma
    y = np.abs(t) + e + delta
    return {'t': t, 'y': y, 'z': z}

train_data = generate_data(n_sample=100)
validation_data = generate_data(n_sample=100)
test_data = generate_data(n_sample=10000)

## Define a PyTorch model $f$ and a moment function $\psi$

In [7]:
model = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(20, 3),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(3, 1)
        )

def moment_function(model_evaluation, y):
    return model_evaluation - y

## Train the model

In [ ]:
trained_model, stats = estimation(model=model,  # Use any PyTorch model
                                  train_data=train_data,    # Format {'t': t, 'y': y, 'z': z}
                                  moment_function=moment_function,  # moment_function(model_eval, y) -> (n_sample, dim_y)
                                  estimation_method='KernelELNeural',   # Method in ['OLS', 'GMM', 'GEL', 'KernelEL', 'KernelMMR', 'SMD', 'KernelVMM', 'NeuralVMM', 'KernelELKernel', 'KernelELNeural', 'KernelFGEL', 'NeuralFGEL']
                                  estimator_kwargs=None,    # Non-default arguments for estimators (default at `kel.default_config.py`)
                                  hyperparams=None,     # Non-default hyperparams for estimators as {name: [val1, ..]}
                                  validation_data=None,     # Format {'t': t, 'y': y, 'z': z}
                                  val_loss_func=None,   # Custom validation loss: val_loss_func(model, validation_data) -> float
                                  verbose=True)
# Make prediction
y_pred = trained_model(torch.Tensor(test_data['t']))

Running:  divergence=chi2, reg_param=0.1
Running:  divergence=chi2, reg_param=0.01
Running:  divergence=chi2, reg_param=0.001
Running:  divergence=chi2, reg_param=0.0001
Running:  divergence=chi2, reg_param=1e-06
Running:  divergence=chi2, reg_param=1e-08
Running:  divergence=kl, reg_param=0.1
Running:  divergence=kl, reg_param=0.01
Running:  divergence=kl, reg_param=0.001
Running:  divergence=kl, reg_param=0.0001
